# Predict $\gamma$ with 1D CNNs

In [1]:
import os

import tensorflow as tf
import tensorflow_addons as tfa

from tensorflow.keras import Input, Model, Sequential
from tensorflow.keras.layers import Conv1D, BatchNormalization, ReLU, MaxPool1D, Dense

from tensorflow.data import Dataset

from tensorflow_addons.layers import AdaptiveAveragePooling1D

from ResNet_1D import ResNet18_1D

import pandas as pd
import numpy as np

import time
from datetime import datetime

## Choose Model

In [2]:
model = ResNet18_1D()
model.summary()

Model: "model_8"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 1000)]            0         
                                                                 
 tf.__operators__.getitem (S  (None, 1000, 1)          0         
 licingOpLambda)                                                 
                                                                 
 conv1d (Conv1D)             (None, 500, 32)           256       
                                                                 
 batch_normalization (BatchN  (None, 500, 32)          128       
 ormalization)                                                   
                                                                 
 re_lu (ReLU)                (None, 500, 32)           0         
                                                                 
 max_pooling1d (MaxPooling1D  (None, 250, 32)          0   

## Read Data Set

In [3]:
df = pd.read_csv('gamma_values.csv', names=['bstring', 'gamma'])
df.head()

,bstring,gamma
0,1001011101001001100110101001001011001011101000...,7.637653
1,1011001001101100100011110000100101001101110111...,7.350031
2,0011000110001101100101100001111101110111110111...,7.058517
3,0101100000100111000110111010101111101111110000...,7.825673
4,0111101111000111101110110110011001011011001101...,6.826776


In [4]:
def bstring_to_floats(bs) :
    '''
    Converts string of 0s and 1s to array of floats
    '''
    return np.array([float(b) for b in bs])

In [5]:
# split up input and output
X = df['bstring'].apply(bstring_to_floats).values
Y = df['gamma'].values

# convert to tensors
X = tf.convert_to_tensor(
    [tf.convert_to_tensor(x) for x in X]
)
Y = tf.convert_to_tensor(Y)

In [6]:
# form data set
ds = Dataset.from_tensor_slices((X, Y))

In [7]:
# split into train/val
train_frac = 0.9
trainN = int(train_frac*len(ds))
valN = len(ds) - trainN
ds = ds.shuffle(100000, seed=42)
train_ds = ds.take(trainN)
val_ds = ds.skip(trainN).take(valN)

# batch up data sets
train_ds = train_ds.batch(64)
val_ds = val_ds.batch(64)

## Train!

In [8]:
# custom loss
def linear_scaled_loss(y, p) :
    return y*(p - y)**2

In [9]:
# choose compile parameters
loss = linear_scaled_loss #tf.keras.losses.MeanSquaredError()
optimizer = tf.keras.optimizers.Adam(learning_rate=1e-5)

# compile
model.compile(loss=loss, optimizer=optimizer)

In [10]:
history = model.fit(
    x=train_ds,
    validation_data=val_ds,
    epochs=30
)

Epoch 1/30
282/282 [==============================] - 71s 221ms/step - loss: 327.0628 - val_loss: 221.5573
Epoch 2/30
282/282 [==============================] - 61s 217ms/step - loss: 126.1809 - val_loss: 95.3469
Epoch 3/30
282/282 [==============================] - 61s 215ms/step - loss: 31.2694 - val_loss: 48.3115
Epoch 4/30
282/282 [==============================] - 61s 215ms/step - loss: 15.8208 - val_loss: 3.3543
Epoch 5/30
282/282 [==============================] - 60s 214ms/step - loss: 16.2661 - val_loss: 28.2806
Epoch 6/30
282/282 [==============================] - 61s 216ms/step - loss: 15.9133 - val_loss: 7.0339
Epoch 7/30
282/282 [==============================] - 60s 211ms/step - loss: 14.1485 - val_loss: 54.4640
Epoch 8/30
282/282 [==============================] - 59s 210ms/step - loss: 15.4913 - val_loss: 6.1324
Epoch 9/30
282/282 [==============================] - 60s 213ms/step - loss: 12.4542 - val_loss: 43.2290
Epoch 10/30
282/282 [==============================] - 

In [11]:
outdir = './models/linear_scaled_loss'
#os.makedirs(outdir)

model.save_weights( os.path.join(outdir, 'checkpoint002') )